In [ ]:
# load data
from cot import Collection
coll = Collection("open_book_qa", load_pregenerated_cots=False)
coll = coll.select(split="test", number_samples=1)
coll

In [ ]:
# load local model

from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [13]:
"""Create a sequential langchain consisting of one for reasoning generation and the 
other for answer extraction. Inspired by https://js.langchain.com/docs/modules/chains/sequential_chain"""
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain


In [21]:
"""Reasoning chain"""

llm = local_llm

template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")

In [22]:
"""answer extraction chain"""

extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="predicted_answer")

In [23]:
"""combination of generation and extraction chain"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "predicted_answer"],
                                verbose=True)

In [ ]:
"""Create an input dictionary to feed the chain"""
input_dict = {'input_dict':
              {
                  'chain': overall_chain,
                  "instruction": "Be faithful and a little hopeful",
                  "cot_trigger": "Answer: Let's think step by step.",
                  "answer_extraction": "Therefore, among A through D, the answer is",
                  'model': "flan-T5-small",
                  'temperature': 0,
                  'max_tokens': 800
              }
              }

"""generate_extract_flexible is called in the dataloade.py file where the collection resides.
 From there a function in the generate.py file is loaded. Next to this function you could use the functions
 {generate/extract/metareason}_flexible"""    
coll.generate_extract_flexible(input_dict=input_dict)

In [36]:
# Evaluate the answer given
coll.evaluate(title='custom_model_title')

  0%|          | 0/1 [00:00<?, ?ex/s]

{'open_book_qa': {'test': {'accuracy': {'gpt-3.5-turbo': {'custom_model_title': 0.0}}}}}